<a href="https://colab.research.google.com/github/rypotter/depo/blob/master/GROQ_RAG_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Eredet author: https://www.linkedin.com/in/mickymultani/

Eredeti a LinkedIn-en:
https://www.linkedin.com/pulse/build-lightning-fast-rag-chatbot-powered-groqs-lpu-ollama-multani-ssloc/

Eredeti Colab: https://colab.research.google.com/drive/1Obrby8RniFfjUvf3DhbNHC6-CmBdiXbY?usp=sharing

Eredeti Github: https://github.com/mickymultani/Groq-RAG

In [ ]:
!pip install groq langchain langchain-core langchain-groq chromadb pypdf gradio

In [ ]:
!pip install lshw

In [ ]:
!curl https://ollama.ai/install.sh | sh

After installing Ollama per the above cell, open the notebook terminal from the bottom left and start ollama by entering `ollama serve`. The following cells with ollama commands ***will only work*** if ollama is running!

Az Ollama fenti cella szerinti telepítése után nyissa meg a notebook terminált a bal alsó sarokban, és indítsa el az ollama-t az ***ollama serve*** beírásával. A következő ollama parancsokat tartalmazó cellák csak akkor működnek, ha az ollama fut!

A notebook terminál csak Colab Pro-ban müxik, ezért itt a ***shellinabox*** terminált használom. Vagy az iframe-ben kell elindítani a terminált, vagy böngésző lapon, és ott beírni a parancsot:  ***ollama serve***




In [ ]:
# Setup shellinabox
!apt install shellinabox &> /dev/null
!nohup shellinaboxd --disable-ssl --no-beep --port=8000 --css /etc/shellinabox/options-enabled/00_White\ On\ Black.css -s "/:root:root:/root:/bin/bash -c bash -i" &> /dev/null &
!yes | /usr/local/sbin/unminimize &> /dev/null

In [ ]:
# Vagy
# Open a terminal in an iframe
from google.colab.output import serve_kernel_port_as_iframe
serve_kernel_port_as_iframe(8000)

In [ ]:
# Vagy
# Open a terminal in a browser tab
from google.colab.output import serve_kernel_port_as_window
serve_kernel_port_as_window(8000, anchor_text = "Open a terminal")

In [ ]:
!ollama pull nomic-embed-text

In [ ]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import embeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFDirectoryLoader
from google.colab import userdata
import os
import time
import textwrap
import gradio as gr

In [ ]:
loader = PyPDFDirectoryLoader("data")
the_text = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(the_text)

In [ ]:
# For text files
# file_path = r'data/YourTextFileName.txt'
# loader = TextLoader(file_path)
# the_text = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
# chunks = text_splitter.split_documents(the_text)

In [ ]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    collection_name="ollama_embeds",
    embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

In [ ]:
%env GROQ_API_KEY=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
#api_key=os.environ.get("GROQ_API_KEY")
api_key=os.environ["GROQ_API_KEY"]
groq_api_key = api_key # userdata.get('groq_api_key')
print(groq_api_key)

In [ ]:
llm = ChatGroq(
            groq_api_key=groq_api_key,
            model_name='mixtral-8x7b-32768'
    )

In [ ]:
rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Test the architecture with a simple hard coded question
response = rag_chain.invoke("What is this document about")
print(textwrap.fill(response, width=80))

In [ ]:
# Make the questions dynamic using a chat interface. Let's use gradio for this.
def process_question(user_question):
    start_time = time.time()

    # Directly using the user's question as input for rag_chain.invoke
    response = rag_chain.invoke(user_question)

    # Measure the response time
    end_time = time.time()
    response_time = f"Response time: {end_time - start_time:.2f} seconds."

    # Combine the response and the response time into a single string
    full_response = f"{response}\n\n{response_time}"

    return full_response

# Setup the Gradio interface
iface = gr.Interface(fn=process_question,
                     inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
                     outputs=gr.Textbox(),
                     title="GROQ CHAT",
                     description="Ask any question about your document, and get an answer along with the response time.")

# Launch the interface
iface.launch()